# Ensemble Clustering for Graphs (ECG)
In this notebook, we will use cuGraph to identify the cluster in a test graph using the Ensemble Clustering for Graph approach.  

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware        |
| --------------|------------|------------------|-----------------|-----------------------|
| Brad Rees     | 04/24/2020 | created          | 0.6             |                       |
|               | 08/16/2020 | updated          | 0.15            | GV100, CUDA 10.2      |
|               | 08/05/2021 | tested/updated   | 21.10 nightly   | RTX 3090 CUDA 11.4    |
| Don Acosta    | 07/20/2022 | tested/updated   | 22.08 nightly   | DGX Tesla V100 CUDA 11.5   |
| Ralph Liu    | 07/26/2022 | updated  | 22.08 nightly   | DGX Tesla V100 CUDA 11.5   |

## Introduction

The Ensemble Clustering for Graphs (ECG) method of community detection is based on the Louvain algorithm

For a detailed description of the algorithm see: https://arxiv.org/abs/1809.05578

It takes as input a cugraph.Graph object and returns as output a 
cudf.Dataframe object with the id and assigned partition for each 
vertex as well as the final modularity score

To compute the ECG cluster in cuGraph use: <br>
   __df = cugraph.ecg(G, min_weight = 0.05 , ensemble_size = 16 )__
   
    Parameters
    ----------
    G cugraph.Graph
        cuGraph graph descriptor, should contain the connectivity information and weights. 
        The adjacency list will be computed if not already present.
    min_weight: floating point
        The minimum value to assign as an edgeweight in the ECG algorithm. 
        It should be a value in the range [0,1] usually left as the default value of .05
    ensemble_size: integer
        The number of graph permutations to use for the ensemble. 
        The default value is 16, larger values may produce higher quality partitions for some graphs.
    
    
    Returns
    -------
    parts : cudf.DataFrame
        A GPU data frame of size V containing two columns the vertex id and the
        partition id it is assigned to.
        
            df[‘vertex’] cudf.Series
                Contains the vertex identifiers
            df[‘partition’] cudf.Series
                Contains the partition assigned to the vertices
        
        
    All vertices with the same partition ID are in the same cluster
        


### References
* Poulin, V., & Théberge, F. (2018, December). Ensemble clustering for graphs. In International Conference on Complex Networks and their Applications (pp. 231-243). Springer, Cham.


#### Some notes about vertex IDs...

* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


<img src="../../img/zachary_graph_clusters.png" width="35%"/>

Because the test data has vertex IDs starting at 1, the auto-renumber feature of cuGraph (mentioned above) will be used so the starting vertex ID is zero for maximum efficiency. The resulting data will then be auto-unrenumbered, making the entire renumbering process transparent to users.


### Prep

In [ ]:
# Import needed libraries
import cugraph
import cudf

# Import a built-in dataset
from cugraph.experimental.datasets import karate

## Create an Edgelist

In [ ]:
# You can also just get the edgelist
gdf = karate.get_edgelist(fetch=True)

# The algorithm also requires that there are vertex weights.  Just use 1.0 
gdf["data"] = 1.0

In [ ]:
# just for fun, let's look at the data types in the dataframe
gdf.dtypes

In [ ]:
# create a Graph - since the data does not start at '0', use the auto-renumbering feature
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst', edge_attr='data', renumber=True)

In [ ]:
# Call ecg on the graph
df = cugraph.ecg(G) 

In [ ]:
df.dtypes

In [ ]:
# How many partitions where found
part_ids = df["partition"].unique()

In [ ]:
print(str(len(part_ids)) + " partition detected")

In [ ]:
# print the clusters.  
for p in range(len(part_ids)):
    part = []
    for i in range(len(df)):
        if (df['partition'].iloc[i] == p):
            part.append(df['vertex'].iloc[i] )
    print("Partition " + str(p) + ":")
    print(part)


___
Copyright (c) 2019-2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___